In [86]:
import kagglehub
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix


import LabelCoder
from collections import Counter
from sklearn.metrics import accuracy_score


path = kagglehub.dataset_download("wenruliu/adult-income-dataset")

In [87]:
data = (pd.read_csv(path+"\\adult.csv",na_values="?",skipinitialspace=True)
        .dropna(how="any")
        )
for col in data.select_dtypes(include='object'):
    data[col] = data[col].str.strip()

print(data['income'].unique())


['<=50K' '>50K']


In [89]:
# 1. workclass: Ordered by employment stability and public sector preference
workclass_map = {
    '?': 0,
    'Without-pay': 1,
    'Never-worked': 2,
    'Self-emp-not-inc': 3,
    'Self-emp-inc': 4,
    'Private': 5,
    'State-gov': 6,
    'Local-gov': 7,
    'Federal-gov': 8
}

# 2. education: Ordered by educational level (from elementary to postgraduate)
education_map = {
    'Preschool': 0,
    '1st-4th': 1,
    '5th-6th': 2,
    '7th-8th': 3,
    '9th': 4,
    '10th': 5,
    '11th': 6,
    '12th': 7,
    'HS-grad': 8,
    'Some-college': 9,
    'Assoc-voc': 10,
    'Assoc-acdm': 11,
    'Bachelors': 12,
    'Masters': 13,
    'Prof-school': 14,
    'Doctorate': 15
}

# 3. marital-status: Ordered by potential socioeconomic stability
marital_status_map = {
    'Never-married': 0,
    'Married-spouse-absent': 1,
    'Separated': 2,
    'Divorced': 3,
    'Widowed': 4,
    'Married-AF-spouse': 5,
    'Married-civ-spouse': 6
}

# 4. occupation: Based on skill level, specialization, and income trends (approximate)
occupation_map = {
    '?': 0,
    'Priv-house-serv': 1,
    'Handlers-cleaners': 2,
    'Other-service': 3,
    'Farming-fishing': 4,
    'Machine-op-inspct': 5,
    'Transport-moving': 6,
    'Craft-repair': 7,
    'Adm-clerical': 8,
    'Sales': 9,
    'Tech-support': 10,
    'Protective-serv': 11,
    'Exec-managerial': 12,
    'Prof-specialty': 13,
    'Armed-Forces': 14
}
income_map = {
    '<=50K': 0, '>50K': 1
}
# 5. relationship: Ordered by likely household responsibility (somewhat subjective)
relationship_map = {
    'Own-child': 0,
    'Other-relative': 1,
    'Not-in-family': 2,
    'Unmarried': 3,
    'Wife': 4,
    'Husband': 5
}

# 6. race: No natural order — included here only symbolically (use with caution)
race_map = {
    'Amer-Indian-Eskimo': 0,
    'Other': 1,
    'Asian-Pac-Islander': 2,
    'Black': 3,
    'White': 4
}

# 7. gender: Binary, Female = 0, Male = 1
gender_map = {
    'Female': 0,
    'Male': 1
}

# 8. native-country: Ordered roughly by economic development or immigration relevance
native_country_map = {
    '?': 0,
    'Honduras': 1,
    'Guatemala': 2,
    'Nicaragua': 3,
    'El-Salvador': 4,
    'Haiti': 5,
    'Dominican-Republic': 6,
    'Trinadad&Tobago': 7,
    'Columbia': 8,
    'Ecuador': 9,
    'Peru': 10,
    'Mexico': 11,
    'Cuba': 12,
    'Jamaica': 13,
    'Outlying-US(Guam-USVI-etc)': 14,
    'Cambodia': 15,
    'Laos': 16,
    'Vietnam': 17,
    'Thailand': 18,
    'Philippines': 19,
    'South': 20,
    'Hong': 21,
    'China': 22,
    'India': 23,
    'Iran': 24,
    'Japan': 25,
    'Taiwan': 26,
    'Poland': 27,
    'Portugal': 28,
    'Hungary': 29,
    'Greece': 30,
    'Yugoslavia': 31,
    'Italy': 32,
    'Ireland': 33,
    'Scotland': 34,
    'Germany': 35,
    'France': 36,
    'England': 37,
    'Holand-Netherlands': 38,
    'Canada': 39,
    'United-States': 40
}
data['income'].value_counts()

income
<=50K    34014
>50K     11208
Name: count, dtype: int64

In [90]:
data['workclass'] = data['workclass'].apply(lambda x: workclass_map.get(x, 0))
data['education'] = data['education'].apply(lambda x: education_map.get(x, 0))
data['marital-status'] = data['marital-status'].apply(lambda x: marital_status_map.get(x, 0))
data['occupation'] = data['occupation'].apply(lambda x: occupation_map.get(x, 0))
data['relationship'] = data['relationship'].apply(lambda x: relationship_map.get(x, 0))
data['race'] = data['race'].apply(lambda x: race_map.get(x, 0))
data['gender'] = data['gender'].apply(lambda x: gender_map.get(x, 0))
data['income'] = data['income'].apply(lambda x: income_map.get(x, 0))
data['native-country'] = data['native-country'].apply(lambda x: native_country_map.get(x, 0))
for column in data.columns:
    #print(column, data[column].unique()[:10])
    data[column] = data[column].astype('int32')


data_train = data.iloc[:30162]
data_test = data.iloc[30162:]

# vectors
X_train = data_train.drop(columns='income').to_numpy()
y_train = data_train['income'].to_numpy()
X_test = data_test.drop(columns='income').to_numpy()
y_test = data_test['income'].to_numpy()


In [92]:
data_train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 30162 entries, 0 to 32589
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   age              30162 non-null  int32
 1   workclass        30162 non-null  int32
 2   fnlwgt           30162 non-null  int32
 3   education        30162 non-null  int32
 4   educational-num  30162 non-null  int32
 5   marital-status   30162 non-null  int32
 6   occupation       30162 non-null  int32
 7   relationship     30162 non-null  int32
 8   race             30162 non-null  int32
 9   gender           30162 non-null  int32
 10  capital-gain     30162 non-null  int32
 11  capital-loss     30162 non-null  int32
 12  hours-per-week   30162 non-null  int32
 13  native-country   30162 non-null  int32
 14  income           30162 non-null  int32
dtypes: int32(15)
memory usage: 2.0 MB


In [93]:
data_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 30162 entries, 0 to 32589
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   age              30162 non-null  int32
 1   workclass        30162 non-null  int32
 2   fnlwgt           30162 non-null  int32
 3   education        30162 non-null  int32
 4   educational-num  30162 non-null  int32
 5   marital-status   30162 non-null  int32
 6   occupation       30162 non-null  int32
 7   relationship     30162 non-null  int32
 8   race             30162 non-null  int32
 9   gender           30162 non-null  int32
 10  capital-gain     30162 non-null  int32
 11  capital-loss     30162 non-null  int32
 12  hours-per-week   30162 non-null  int32
 13  native-country   30162 non-null  int32
 14  income           30162 non-null  int32
dtypes: int32(15)
memory usage: 2.0 MB


In [97]:
data_train['income'].value_counts()

income
0    22729
1     7433
Name: count, dtype: int64

In [98]:
decisionTree= DecisionTreeClassifier(random_state=42)
decisionTree.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [99]:
decisionTreePruned = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
)

In [100]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Dokładność: {accuracy:.4f}")
    print(f"Raport klasyfikacji:")
    print(classification_report(y_test, y_pred, target_names=['<=50K', '>50K']))

    return y_pred, accuracy

In [101]:
y_pred_basic, acc_basic = evaluate_model(decisionTree, X_test, y_test)
y_pred_pruned, acc_pruned = evaluate_model(decisionTreePruned, X_test, y_test)

Dokładność: 0.8106
Raport klasyfikacji:
              precision    recall  f1-score   support

       <=50K       0.88      0.87      0.87     11285
        >50K       0.62      0.64      0.63      3775

    accuracy                           0.81     15060
   macro avg       0.75      0.75      0.75     15060
weighted avg       0.81      0.81      0.81     15060



NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.